### Set up

`conda install -n bcg-env ipykernel --update-deps --force-reinstall`

### Data importation

In [26]:
import pandas as pd
from random import sample
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from math import ceil
import datetime
import os

In [2]:
root_path = os.path.dirname(os.getcwd())
os.chdir(root_path)

In [3]:
df = pd.read_parquet(os.path.join(root_path, 'data/transaction_data.parquet'),\
    engine='pyarrow')
print(df.shape)
df.head()

(63319315, 8)


,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051


In [13]:
df[["quantity", "order_channel"]].groupby('order_channel').sum()

,quantity
order_channel,
at the store,1583990440
by phone,3587646875
during the visit of a sales rep,3164479
online,660310222
other,179661


### Shorten dataset 
We randomnly sample 1000 clients for the dev part.

In [9]:
n_client = 1000

id_list_short = sample(list(df.client_id.unique()), n_client)
df_short = df[df.client_id.isin(id_list_short)]

print(df_short.shape)
df_short.head()

(424254, 8)


,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
364,2017-09-25,2017-09-25,1449738,1272143,9.3288,3,at the store,5297
883,2017-09-26,2017-09-26,653092,1700424,25.8888,3,by phone,1877
991,2017-09-27,2017-09-27,3115653,1238765,33.1200,9,online,10964
1066,2017-09-29,2017-09-29,770261,1695561,57.9600,3,at the store,5274
1094,2017-09-26,2017-09-26,926970,1798523,868.8480,3,by phone,9700


In [10]:
# Save
df_short.to_csv('data/transaction_data_short.csv', index=False)

In [35]:
# Load
df_short = pd.read_csv('data/transaction_data_short.csv')
df_short.head(2)

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,1449738,1272143,9.3288,3,at the store,5297
1,2017-09-26,2017-09-26,653092,1700424,25.8888,3,by phone,1877


In [40]:
# Sample a client
df_client_sample = df_short[df_short.client_id==1272143]

In [48]:
datetime.datetime(year=2020,)

TypeError: function missing required argument 'month' (pos 2)

In [52]:
# Retrieve the right time feature
to_plot = df_client_sample.copy()
to_plot["date_order"] = pd.to_datetime(to_plot.date_order)
to_plot["year/week"] = to_plot.date_order.apply(lambda x: f'{x.year}/{x.week}')
#to_plot["final_date"] = pd.to_datetime(to_plot["year/week"], "%Y/U")
to_plot.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,week,year,year/week
0,2017-09-25,2017-09-25,1449738,1272143,9.32880,3,at the store,5297,39,2017,2017/39
14,2017-09-25,2017-09-25,683924,1272143,45.78288,11,at the store,5297,39,2017,2017/39
17,2017-09-25,2017-09-25,805106,1272143,32.40700,5,at the store,5297,39,2017,2017/39
38,2017-09-26,2017-09-26,2541918,1272143,13.43936,11,at the store,5297,39,2017,2017/39
67,2017-09-25,2017-09-25,519804,1272143,10.84680,9,at the store,5297,39,2017,2017/39


In [56]:
to_plot2 = to_plot.copy()
to_plot2 = to_plot2[["year/week", "quantity", "order_channel", "sales_net"]]\
    .groupby(["order_channel", "year/week"])\
    .sum()
to_plot2 = to_plot2.reset_index()
to_plot2.head()

,order_channel,year/week,quantity,sales_net
0,at the store,2017/39,902,1908.207591
1,at the store,2017/40,1773,1776.533769
2,at the store,2017/41,3736,4447.077473
3,at the store,2017/42,1074,2906.982604
4,at the store,2017/43,3991,2429.484813


In [62]:
dimension = "sales_net"

fig = go.Figure()
for channel in to_plot2.order_channel.unique():
    x = to_plot2[to_plot2.order_channel==channel]["year/week"]
    y = to_plot2[to_plot2.order_channel==channel][dimension]
    fig.add_trace(go.Scatter(x=x, y=y, mode="markers", name=channel))

fig.update_layout(yaxis_title=dimension, width=800, height=400)
fig.update_layout(xaxis_title='Date (aggregated per week)', width=800, height=400)

fig.show()

### Process the data

We create a new table with features per client and binary target column churn.

In [17]:
client_id_list = list(df_short[["client_id"]].sort_values('client_id').client_id.unique())

In [21]:
churn_table = pd.DataFrame(data={\
    'client_id': client_id_list,\
    'churn': [0 for k in range(len(client_id_list))]
    })
churn_table.head(2)

,client_id,churn
0,60,0
1,774,0
